<a href="https://colab.research.google.com/github/elizabethavargas/Dataset-Description-Generation/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Dataset

In [1]:
# packages
import random
import requests
import pandas as pd
from io import BytesIO

In [ ]:
https://data.cityofnewyork.us/api/v3/views/8wbx-tsch/query.json

https://data.cityofnewyork.us/api/v3/views/8wbx-tsch/query.json?$$app_token=L76aBvmvvFwme9Q46GQJ3qtf8

#PaintThinner3!

### Get list of datasets on NYC Open Data

In [2]:
# Base URL for the NYC Open Data Socrata API
base_url = "https://data.cityofnewyork.us/api/views.json"

try:
    response = requests.get(base_url)
    response.raise_for_status()  # Raise an exception for bad status codes
    datasets_data = response.json()

    # Extract id and name for each dataset
    datasets_list = []
    for dataset in datasets_data:
        if 'id' in dataset and 'name' in dataset:
            datasets_list.append({'id': dataset['id'], 'name': dataset['name']})

    # Print confirmation message
    print(f"Successfully listed {len(datasets_list)} datasets.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except ValueError:
    print("Error decoding JSON response. The response might not be in JSON format.")

Successfully listed 3008 datasets.


In [16]:
# Randomly select 10 datasets
num_datasets_to_query = 10
selected_datasets = random.sample(datasets_list, num_datasets_to_query)
selected_datasets

[{'id': 'g9wv-7n2m',
  'name': '2016-17 Physical Education - MTI All- Star Schools'},
 {'id': 'aizm-q3sx', 'name': 'Newsrack Inspections'},
 {'id': '3t6s-yb67',
  'name': 'Verified Locations for NYC City-Funded Social Service Contracts - Programs'},
 {'id': 'r3dx-pew9',
  'name': 'Neighborhood Financial Health Digital Mapping and Data Tool'},
 {'id': 'y4nf-25nw', 'name': 'VZV_Neighborhood Slow Zones'},
 {'id': '2hrw-qfsu', 'name': 'Detention Admissions by Community District'},
 {'id': '2iga-a6mk', 'name': 'Zoning Application Portal (ZAP) - BBL'},
 {'id': 'hy4q-igkk', 'name': '311 Service Requests for 2006'},
 {'id': 'basd-2jwn', 'name': 'Official Fundraising by City Agencies'},
 {'id': 'x9i6-ckbm', 'name': 'ECS Conduit Data'}]

In [4]:
from difflib import SequenceMatcher

def find_closest_sheet_name(sheet_names, target_names):
    """
    Finds the sheet name in a list that is most similar to one of the target names.

    Args:
        sheet_names (list): A list of sheet names (strings).
        target_names (list): A list of target names (strings) to compare against.

    Returns:
        str: The sheet name from sheet_names that is most similar to a target_name,
             or None if no sheet names are provided.
    """
    if not sheet_names:
        return None

    best_match = None
    highest_similarity = -1

    for sheet_name in sheet_names:
        for target_name in target_names:
            similarity = SequenceMatcher(None, sheet_name.lower(), target_name.lower()).ratio()
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = sheet_name

    return best_match



In [13]:
import pandas as pd
import json

def format_example_rows(df, max_chars=150):
    """
    Formats the first n rows of a DataFrame as clean JSON-like examples for LLM prompts.
    - Truncates long strings/numbers.
    - Removes NaN values.
    """
    example = {}
    for col, val in df.iloc[0].items():
        # Simplify nested dicts or long text
        if isinstance(val, (dict, list)):
            val_str = json.dumps(val)
        else:
            val_str = str(val)
        if len(val_str) > max_chars:
            val_str = val_str[:max_chars] + "..."
        if val_str not in ("nan", "None", ""):
            example[col] = val_str
    formatted = f"Example row: {json.dumps(example, ensure_ascii=False, indent=2)}\n"
    return formatted

In [17]:
prompts = []
for dataset in selected_datasets:
    dataset_id = dataset['id']
    dataset_name = dataset['name']
    print(f"Querying dataset: {dataset_name} (ID: {dataset_id}) ---")

    # Construct the API endpoint URL for the dataset and metadata
    dataset_url = f"https://data.cityofnewyork.us/resource/{dataset_id}.json" #f"https://data.cityofnewyork.us/api/v3/views/{dataset_id}/query.json?$$app_token=L76aBvmvvFwme9Q46GQJ3qtf8" #f"https://data.cityofnewyork.us/resource/{dataset_id}.json"
    metadata_url = f"https://data.cityofnewyork.us/api/views/{dataset_id}.json"


    try:
        print("Fetching data...")
        data_response = requests.get(dataset_url, headers={"X-App-Token": "L76aBvmvvFwme9Q46GQJ3qtf8"}, params={'$limit': 2})

        data_response.raise_for_status()  # Raise an exception for bad status codes
        data = data_response.json()

        if data:
            df = pd.DataFrame(data)
            print(f"Successfully retrieved data for {dataset_name}:")
            display(df.head())
        else:
            print(f"No data returned for dataset: {dataset_name}")


    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {dataset_name}: {e}")
    except ValueError:
        print(f"Error decoding JSON response for data of {dataset_name}. The response might not be in JSON format.")
    except Exception as e:
        print(f"An unexpected error occurred while processing data for {dataset_name}: {e}")

    try:
        print("Fetching metadata...")
        metadata_response = requests.get(metadata_url)
        metadata_response.raise_for_status() # Raise an exception for bad status codes
        metadata = metadata_response.json()

        if metadata:
            # Get category, description, agency, and tags
            category = metadata.get('category', 'N/A')
            description = metadata.get('description', 'N/A')
            agency = metadata.get('metadata', {}).get('custom_fields', {}).get('Dataset Information', {}).get('Agency', 'N/A')
            tags = metadata.get('tags', [])
            title = metadata.get('name', 'N/A') # Assuming 'name' is the title

            prompt = f"Write an improved dataset description for the dataset titled {title} with the category {category}, and by the agency {agency}."

            if tags:
              prompt += f" The tags are {tags}."
            if description:
              cleaned_description = "\n".join([line for line in description.splitlines() if line.strip()])
              prompt += f""" The current description is "{cleaned_description}"."""

            prompt += format_example_rows(df)

            # add column descriptions
            for column in metadata.get('columns'):
              column_name = column.get('name', 'N/A')
              column_description = column.get('description', '')
              if column_description != '':
                prompt += f"{column_name}: {column_description}\n"

            print(f"Generated prompt:\n{prompt}")
            prompts.append(prompt)


            # --- Data Dictionary ---
            attachments = metadata.get('metadata', {}).get('attachments', [])
            if attachments:
                for attach in attachments:
                    file_id = attach.get('assetId')
                    filename = attach.get('filename', 'data_dictionary.xlsx')
                    if file_id:
                        file_url = f"https://data.cityofnewyork.us/api/views/{dataset_id}/files/{file_id}?download=true&filename={filename}"
                        print(f"Downloading data dictionary: {filename} ...")
                        try:
                            file_response = requests.get(file_url)
                            file_response.raise_for_status()
                            excel_file = BytesIO(file_response.content)
                            xls = pd.ExcelFile(excel_file)
                            print(f"Sheets in data dictionary: {xls.sheet_names}")
                            target_names = ["data dictionary", "column descriptions", "column definitions"]
                            closest_sheet = find_closest_sheet_name(xls.sheet_names, target_names)
                            print("First few rows of data dictionary:")
                            try:
                                # Attempt to read the sheet with the default header (row 0)
                                df_dict = pd.read_excel(xls, sheet_name=closest_sheet)

                                # Check if a significant number of columns are unnamed
                                unnamed_columns = [col for col in df_dict.columns if 'Unnamed:' in col]
                                if len(unnamed_columns) > len(df_dict.columns) * 0.5 and df_dict.shape[0] > 1:
                                    print(f"Potential issue with header in sheet '{closest_sheet}'. Attempting to use the next row as header.")
                                    # Read the sheet again, using the second row (index 1) as the header
                                    df_dict = pd.read_excel(xls, sheet_name=closest_sheet, header=1)

                                print(f"First few rows of data dictionary from sheet '{closest_sheet}':")
                                display(df_dict.head())

                            except Exception as e:
                                print(f"Error reading sheet '{closest_sheet}': {e}")

                        except Exception as e:
                            print(f"Error opening data dictionary: {e}")
                    else:
                        print(f"No file ID found for {filename}")
            else:
                print("No data dictionary attachment found.")
        else:
            break
            print(f"No metadata returned for dataset: {dataset_name}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching metadata for {dataset_name}: {e}")
    except ValueError:
        print(f"Error decoding JSON response for metadata of {dataset_name}. The response might not be in JSON format.")
    except Exception as e:
        print(f"An unexpected error occurred while processing metadata for {dataset_name}: {e}")

Querying dataset: 2016-17 Physical Education - MTI All- Star Schools (ID: g9wv-7n2m) ---
Fetching data...
Successfully retrieved data for 2016-17 Physical Education - MTI All- Star Schools:


,dbn,school_name_lcgms
0,01M539,"New Explorations into Science, Technology and ..."
1,02M006,P.S. 006 Lillie D. Blake


Fetching metadata...
Generated prompt:
Write an improved dataset description for the dataset titled 2016-17 Physical Education - MTI All- Star Schools with the category Education, and by the agency Department of Education (DOE). The tags are ['2016', '2017', 'physical education', 'mti', 'all- star schools', 'reference code jseqjq']. The current description is "N/A".Example row: {
  "dbn": "01M539",
  "school_name_lcgms": "New Explorations into Science, Technology and Math High School"
}

Sheets in data dictionary: ['Dataset Info', 'Column Info', 'Dataset Revision History', 'Sheet2']
First few rows of data dictionary:
Potential issue with header in sheet 'Column Info'. Attempting to use the next row as header.
First few rows of data dictionary from sheet 'Column Info':


,Column Name,Column Description,"Term, Acronym, or Code Definitions","Additional Notes \n(where applicable, include the range of possible values, units of measure, how to interpret null/zero values, whether there are specific relationships between columns, and information on column source)"
0,DBN,DBN CODE: a character code used to identify a ...,"the district, borough and number for a school",NaN
1,School Name (LCGMS),Official School Name,LCGMS system (Location Name),NaN


Querying dataset: Newsrack Inspections (ID: aizm-q3sx) ---
Fetching data...
Successfully retrieved data for Newsrack Inspections:


...

,Column Name,Column Description,Expected/Allowed Values,Field Limitations,Additional Notes
0,Name of the column exactly as it appears in th...,"A brief, plain-language explanation of what th...",Specifies if there is an expected range and/or...,Describes any unique characteristics or potent...,Provides any additional relevant information a...
1,Year,Calendar year during which the donation was made.,NaN,NaN,NaN
2,Agency Name,Name of City agency reporting donations.,NaN,NaN,NaN
3,Name of Not-for-Profit,Name of not-for-profit organization that was t...,NaN,NaN,Agencies are required to report the not-for-pr...


Querying dataset: ECS Conduit Data (ID: x9i6-ckbm) ---
Fetching data...
Successfully retrieved data for ECS Conduit Data:


,nta,nta_name,borough,boro_code,company,conduit_route_mileage,street_mileage,conduit_mileage_total,conduit_mileage_available,conduit_mileage_available_1,cd_overlap_1,cd_overlap_2
0,BX01,Claremont-Bathgate,Bronx,2,ECS,12,27.44,269.6,262.6,0.974,203,206
1,BX03,Eastchester-Edenwald-Baychester,Bronx,2,ECS,5.3,46.17,65.3,63.1,0.966,212,NaN


Fetching metadata...
Generated prompt:
Write an improved dataset description for the dataset titled ECS Conduit Data with the category Business, and by the agency Office of Technology and Innovation (OTI). The tags are ['fiber', 'conduit', 'broadband', 'internet']. The current description is "Data about Empire City Subway’s (ECS) conduit for fiber-optic and other cable, aggregated at the Neighborhood Tabulation Area (NTA) level.".Example row: {
  "nta": "BX01",
  "nta_name": "Claremont-Bathgate",
  "borough": "Bronx",
  "boro_code": "2",
  "company": "ECS",
  "conduit_route_mileage": "12",
  "street_mileage": "27.44",
  "conduit_mileage_total": "269.6",
  "conduit_mileage_available": "262.6",
  "conduit_mileage_available_1": "0.974",
  "cd_overlap_1": "203",
  "cd_overlap_2": "206"
}
NTA: Four (4) digit code assigned to neighborhoods as created by the NYC Department of City Planning using whole census tracts from the 2010 Census as building blocks. These aggregations of census tracts a

,Column Name,Column Description,"Term, Acronym, or Code Definitions","Additional Notes \n(where applicable, include the range of possible values, units of measure, how to interpret null/zero values, whether there are specific relationships between columns, and information on column source)"
0,NTA,Neighborhood Tabulation Area #,Four (4) digit code assigned to neighborhoods ...,plain text
1,NTA_name,Neighborhood Tabulation Area Name,Name assigned to neighborhoods as created by t...,plain text
2,borough,Borough,The New York City borough where the conduits a...,plain text
3,boro_code,Borough Code,One (1) digit code assigned to differentiate N...,plain text
4,company,Company Name,name of the company that holds an agreement wi...,plain text


In [18]:
for prompt in prompts:
  print(prompt)

Write an improved dataset description for the dataset titled 2016-17 Physical Education - MTI All- Star Schools with the category Education, and by the agency Department of Education (DOE). The tags are ['2016', '2017', 'physical education', 'mti', 'all- star schools', 'reference code jseqjq']. The current description is "N/A".Example row: {
  "dbn": "01M539",
  "school_name_lcgms": "New Explorations into Science, Technology and Math High School"
}

Write an improved dataset description for the dataset titled Newsrack Inspections with the category Transportation, and by the agency Department of Transportation (DOT). The tags are ['newsrack', 'news rack', 'inspection', 'nis']. The current description is "Tracks complaints, inspections and reinspections of News racks and reports on the Inspection results.".Example row: {
  "newsracklocationid": "14171",
  "boroughname": "Queens",
  "publishername": "Schneps Media , LLC",
  "publicationname": "Queens Family",
  "newsrackstatus": "Inspecte